# Import packages and data 

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy.stats
import anndata
import matplotlib.pyplot as plt
import matplotlib as mpl
import rpy2

In [2]:
sc.settings.verbosity = 1  # verbosity: errors (0), warnings (1), info (2), hints (3)
# Set up the plot config for viewing the annotation clearly.
sc.settings.set_figure_params(dpi=120, dpi_save=1000)
sc.logging.print_versions()

scanpy==1.4.4 anndata==0.7.1 umap==0.3.10 numpy==1.17.1 scipy==1.4.1 pandas==0.23.4 scikit-learn==0.22.2.post1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Save UMAP for Monocle3 from the processed dataset

In [4]:
adata = sc.read("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/data/fig5e_5f_downs_eryth_dr_20200625.h5ad")

In [5]:
# save the info on variable genes used in scanpy so these can be transferred to monocle
gene_df = adata.var

In [6]:
pd.DataFrame(adata.obsm["X_draw_graph_fa"]).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/downs_eryth_fdg_20200625.csv")

# Load the fetal BM cells from the raw dataset

In [7]:
adata = sc.read('/Users/b8058304/Documents/PhD_work/Coding/downs_bm/data/downs_raw_dr_20200311.h5ad')

In [8]:
adata.raw = adata

In [9]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CAR                          4
CD8 T cell                  55
CD14 monocyte              252
CD56 bright NK              36
CMP                         37
DC1                         16
DC2                         94
DC3                         98
HSC                         45
ILC precursor               13
MEMP                        42
MK                          25
MSC                         18
Treg                         8
chondrocyte                  4
early B                     23
early MK                     7
early erythroid            766
endothelium                 37
eo/baso/mast precursor      40
eosinophil                  41
late erythroid            3341
macrophage                  64
mast cell                   27
mature B                    20
mature NK                   69
mid erythroid             2082
monocyte precursor         172
myelocyte                  218
neut-myeloid prog.          27
neutrophil                 245
neutrophil precursor       

In [10]:
celltypes = ['HSC', 'MEMP', 'early erythroid']

In [11]:
adata = adata[adata.obs['cell.labels'].isin(celltypes)].copy()

In [12]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
HSC                 45
MEMP                42
early erythroid    766
dtype: int64

In [13]:
adata.var = gene_df

In [14]:
adata.shape

(853, 33694)

# Convert to CDS

# Load rpy2 link

In [15]:
# convert anndata to sce
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [16]:
%R d = as.data.frame( R.Version() )
%R d = d['version.string']

,version.string
1,R version 3.6.2 (2019-12-12)


In [17]:
%%R 

library(scran)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(clusterExperiment)
library(ggplot2)
library(plyr)

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: GenomicRanges

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find

# Convert Scanpy object to CellDataSet object and prepare obj

In [18]:
%%R -i adata
adata

class: SingleCellExperiment 
dim: 33694 853 
metadata(0):
assays(1): X
rownames(33694): RP11-34P13.3 FAM138A ... AC213203.1 FAM231B
rowData names(5): gene_ids highly_variable means dispersions
  dispersions_norm
colnames(853): AAACCTGAGTACGTTC-1-DSOX4 AAAGATGTCTTATCTG-1-DSOX4 ...
  TTTGCGCTCTGGGCCA-1-DSOX19_2 TTTGTCATCTAGCACA-1-DSOX19_2
colData names(8): lanes mad_prd ... age percent.mito
reducedDimNames(3): X_orig_pca PCA UMAP
spikeNames(0):
altExpNames(0):


In [19]:
#Preprocessing for monocle
data_mat_mon = adata.X.T
var_mon=adata.var.copy()
obs_mon=adata.obs.copy()

In [20]:
var_mon

,gene_ids,highly_variable,means,dispersions,dispersions_norm
index,,,,,
RP11-34P13.3,ENSG00000243485,False,9.124730e-03,0.388069,0.216294
FAM138A,ENSG00000237613,False,1.000000e-12,NaN,NaN
OR4F5,ENSG00000186092,False,1.000000e-12,NaN,NaN
RP11-34P13.7,ENSG00000238009,False,5.019189e-03,-0.270088,-0.911539
RP11-34P13.8,ENSG00000239945,False,1.000000e-12,NaN,NaN
RP11-34P13.14,ENSG00000239906,False,1.000000e-12,NaN,NaN
RP11-34P13.9,ENSG00000241599,False,1.000000e-12,NaN,NaN
FO538757.3,ENSG00000279928,False,1.192551e-02,-0.356099,-1.058929
FO538757.2,ENSG00000279457,False,1.283056e+00,0.467465,-0.428835


In [21]:
%%R -i data_mat_mon -i obs_mon -i var_mon

#Set up the CellDataSet data structure
print("setting up cell dataset structure")
print("printing pd - cells")
pd <- AnnotatedDataFrame(data = obs_mon)
print(pd)
print("printing fd - genes")
fd <- AnnotatedDataFrame(data = var_mon)
print(fd)
# assigning cells as column name and genes as rownames of GEX matrix (data_mat_mon)
colnames(data_mat_mon) <- rownames(pd)
rownames(data_mat_mon) <- rownames(fd)
print("create newCellDataSet using the matrix, cells and genes")
ie_regions_cds <- newCellDataSet(cellData=data_mat_mon, phenoData=pd, featureData=fd, expressionFamily=negbinomial.size())

# print ie_regions_cds
print("printing ie_regions_cds")
print(ie_regions_cds)

[1] "setting up cell dataset structure"
[1] "printing pd - cells"
An object of class 'AnnotatedDataFrame'
  rowNames: AAACCTGAGTACGTTC-1-DSOX4 AAAGATGTCTTATCTG-1-DSOX4 ...
    TTTGTCATCTAGCACA-1-DSOX19_2 (853 total)
  varLabels: lanes mad_prd ... percent.mito (8 total)
  varMetadata: labelDescription
[1] "printing fd - genes"
An object of class 'AnnotatedDataFrame'
  rowNames: RP11-34P13.3 FAM138A ... FAM231B (33694 total)
  varLabels: gene_ids highly_variable ... dispersions_norm (5 total)
  varMetadata: labelDescription
[1] "create newCellDataSet using the matrix, cells and genes"
[1] "printing ie_regions_cds"
CellDataSet (storageMode: environment)
assayData: 33694 features, 853 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: AAACCTGAGTACGTTC-1-DSOX4 AAAGATGTCTTATCTG-1-DSOX4 ...
    TTTGTCATCTAGCACA-1-DSOX19_2 (853 total)
  varLabels: lanes mad_prd ... Size_Factor (9 total)
  varMetadata: labelDescription
featureData
  featureNames: RP11-34P13.3 FAM138A .

# Within CDS, select HVGs (previously defined in scanpy)

In [22]:
%%R 

#Filter highly variable genes from our analysis (normalisation by log and size factor will be performed in monocle3)
print("filter hvgs")
hvg_mask = fData(ie_regions_cds)$highly_variable
ie_regions_cds <- ie_regions_cds[hvg_mask,]

# print ie_regions_cds
print("printing cds again")
print(ie_regions_cds)

[1] "filter hvgs"
[1] "printing cds again"
CellDataSet (storageMode: environment)
assayData: 2634 features, 853 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: AAACCTGAGTACGTTC-1-DSOX4 AAAGATGTCTTATCTG-1-DSOX4 ...
    TTTGTCATCTAGCACA-1-DSOX19_2 (853 total)
  varLabels: lanes mad_prd ... Size_Factor (9 total)
  varMetadata: labelDescription
featureData
  featureNames: RP4-669L17.10 ISG15 ... MT-ND6 (2634 total)
  fvarLabels: gene_ids highly_variable ... dispersions_norm (5 total)
  fvarMetadata: labelDescription
experimentData: use 'experimentData(object)'
Annotation:  


# Save object for input into Monocle3

In [23]:
%%R 

saveRDS(ie_regions_cds, 
        "/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/data/fig5e_5f_downs_eryth_cds_20200625.RDS")